<a href="https://colab.research.google.com/github/mlister3/Analysis-Crowdfunding-Excel-Workbook/blob/main/PDF_Mail_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Admissions Certification PDF Merger

- This program downloads the blank certificate, asks for the excel query used for the admit letters and automatically merges, generates, and downloads all the certificates as PDFs.

The latest query for 2024 Freshman and Transfer can be found [here](https://apply.r.umn.edu/manage/query/query?id=afc0ee11-90ff-412a-8924-8ab1cb44a9f1)

- - -

Instructions:

1. When you hover your mouse over this instructions box, a play button will appear. Press the play button

2. Hover your mouse over the green text below that says "Run Program"

3. When prompted, select the MS Excel sheet recieved from the SLATE query used for the Admissions letter.

4. The PDFs will automatically download as a .zip file.

In [1]:
print("... Loading required dependencies")
!pip install PyPDF2 -q
import PyPDF2 as pdf2
import requests
import os
import time
from google.colab import files
import pandas as pd
import zipfile

pdf_url = "https://drive.google.com/uc?export=download&id=1nGBWdZ7lBO2tv_OsQrOP6iXS7BsoV5di" # Link to blank Admissions Certificate
save_path = "Acceptance Certificate 2024 Form.pdf"

... Loading required dependencies
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [2]:
# Downloader function
def download_pdfs(debug_mode, url, save_path):
    if debug_mode != 0:
      print("RUNNING IN DEBUG MODE")
    print("... Downloading Admissions Certificate from Admissions Google Drive")
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"PDF downloaded and saved to {save_path}")
        get_excel(debug_mode)
    elif debug_mode != 0:
        print(f"Failed to download PDF from {url}, Status code: {response.status_code}")
    else:
        print("Failed to download PDF")

In [3]:
def get_excel(debug_mode):
  # Get download path
  current_directory = os.getcwd()

  # Get download path for PDF
  pdf_path = os.path.join(current_directory, save_path)

  print("... Preparing to recieve merging Excel File")
  print("Upload Excel file by clicking the button below: VVV")
  while True:
    try:
      # Upload list
      mail_list_upload = files.upload()

      file_name, file_content = next(iter(mail_list_upload.items()))
      # Write the content to a file
      with open(file_name, 'wb') as f:
        f.write(file_content)
      mail_list_df = pd.read_excel(file_name)
      break
    except ValueError:
      print("No file selected or error occurred.")
    except NameError:
      print("Press play on first box before pressing this one.")

  print("Verify upload integrity by reviewing the column names in the frame below.")
  mail_list_columns = mail_list_df.columns
  for column in mail_list_columns:
    print("... column " + column)

  generate_admits(debug_mode, mail_list_df, pdf_path)

In [4]:
def generate_admits(debug_mode, mail_list_df, pdf_path):
  # Calculate number of files that will be generated
  max_index = len(mail_list_df)
  zip_list = []
  print(f"... The list contains {max_index} student(s) where Admissions Certificates will be generated")

  # Open the PDF file
  with open(pdf_path, 'rb') as source:
    reader = pdf2.PdfReader(source)
    #final_merger = pdf2.PdfMerger()

    for admitted_student, row in mail_list_df.iterrows():
      name_index = str(row['Person Preferred'] + " " + row['Person Last'])
      various_fields = {'#FutureRaptor': name_index}
      writer = pdf2.PdfWriter()
      writer.append_pages_from_reader(reader, lambda admitted_student: writer.update_page_form_field_values(admitted_student, various_fields, 1))
      save_name_individual = str(name_index) + " Certificate.pdf"
      zip_list.append(save_name_individual)
      with open(save_name_individual, 'wb') as output_pdf_file:
        writer.write(output_pdf_file)
        writer.close()

  zip_n_download(zip_list, debug_mode)

In [5]:
def zip_n_download(zip_list, debug_mode):
  # Create a zip file and add PDFs to it
  with zipfile.ZipFile('Completed Admit Certificates.zip', 'w') as zipf:
    for pdf_file in zip_list:
      zipf.write(pdf_file, os.path.basename(pdf_file))

  # Move the zip file to a downloadable location
  os.rename('Completed Admit Certificates.zip', '/content/Completed Admit Certificates.zip')
  files.download('Completed Admit Certificates.zip')

## Run Merger

In [6]:
# Run Program
debug_mode = 0
download_pdfs(debug_mode, pdf_url, save_path)

... Downloading Admissions Certificate from Admissions Google Drive
PDF downloaded and saved to Acceptance Certificate 2024 Form.pdf
... Preparing to recieve merging Excel File
Upload Excel file by clicking the button below: VVV


Saving 2024 Admissions Letter (New Freshman & Transfer) 20240124-162310.xlsx to 2024 Admissions Letter (New Freshman & Transfer) 20240124-162310.xlsx
Verify upload integrity by reviewing the column names in the frame below.
... column Person Preferred
... column Person First
... column Person Last
... column Person Birthdate
... column Addresses Street Combined
... column Addresses City
... column Addresses Region
... column Addresses Postal
... column Person Student ID number
... column Schools Name
... column Person Email
... column Person Mobile Phone
... column Person Text: Opt Out
... column Person Influencer Scholarship Intro
... column Person Influencer Scholarship Auto Value
... column Person Influencer Scholarship Closing
... column Application Applicant Type
... column Application Application Term
... The list contains 58 student(s) where Admissions Certificates will be generated


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>